# • Introduction

<a href="https://colab.research.google.com/github/Nixtla/hierarchicalforecast/blob/main/nbs/examples/Introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Hierarchical Series

![image.png](./imgs/hierarchical_motivation1.png)

![image.png](./imgs/hierarchical_motivation2.png)

## 2. Hierarchical Forecast

## 3. Example

In [ ]:
%%capture
!pip install hierarchicalforecast
!pip install -U numba statsforecast datasetsforecast

### Wrangling Data

In [1]:
import numpy as np
import pandas as pd

# compute base forecast no coherent
from statsforecast.core import StatsForecast
from statsforecast.models import AutoARIMA, Naive

#obtain hierarchical reconciliation methods and evaluation
from hierarchicalforecast.utils import aggregate
from hierarchicalforecast.methods import BottomUp, TopDown
from hierarchicalforecast.core import HierarchicalReconciliation

In [3]:
# Create Figure 1. synthetic data
ds = np.arange(1,9)
r1 = ds * (10**1)
r2 = ds * (10**1)
r3 = ds * (10**2)
r4 = ds * (10**2)

ys = np.concatenate([r1, r2, r3, r4])
ds = np.tile(ds, 4)
unique_ids = ['r1'] * 8 + ['r2'] * 8 + ['r3'] * 8 + ['r4'] * 8
top_level = 'Australia'
middle_level = ['State1'] * 16 + ['State2'] * 16
bottom_level = unique_ids

bottom_df = dict(unique_id=unique_ids, ds=ds, y=ys,
                 top_level=top_level, 
                 middle_level=middle_level, 
                 bottom_level=bottom_level)
bottom_df = pd.DataFrame(bottom_df)
bottom_df.groupby('unique_id').head(2)

unique_id  ds    y  top_level middle_level bottom_level
0         r1   1   10  Australia       State1           r1
1         r1   2   20  Australia       State1           r1
8         r2   1   10  Australia       State1           r2
9         r2   2   20  Australia       State1           r2
16        r3   1  100  Australia       State2           r3
17        r3   2  200  Australia       State2           r3
24        r4   1  100  Australia       State2           r4
25        r4   2  200  Australia       State2           r4

In [6]:
# Create hierarchical structure and constraints
hierarchy_levels = [['top_level'],
                    ['top_level', 'middle_level'],
                    ['top_level', 'middle_level', 'bottom_level']]
Y_hier_df, S_df, tags = aggregate(df=bottom_df, spec=hierarchy_levels)
Y_hier_df.groupby('unique_id').head(2)

ds    y
unique_id                   
Australia             1  220
Australia             2  440
Australia/State1      1   20
Australia/State1      2   40
Australia/State2      1  200
Australia/State2      2  400
Australia/State1/r1   1   10
Australia/State1/r1   2   20
Australia/State1/r2   1   10
Australia/State1/r2   2   20
Australia/State2/r3   1  100
Australia/State2/r3   2  200
Australia/State2/r4   1  100
Australia/State2/r4   2  200

In [7]:
S_df

Australia/State1/r1  ...  Australia/State2/r4
Australia                            1.0  ...                  1.0
Australia/State1                     1.0  ...                  0.0
Australia/State2                     0.0  ...                  1.0
Australia/State1/r1                  1.0  ...                  0.0
Australia/State1/r2                  0.0  ...                  0.0
Australia/State2/r3                  0.0  ...                  0.0
Australia/State2/r4                  0.0  ...                  1.0

[7 rows x 4 columns]

### Base Predictions

### Reconciliation

## References

- [Hyndman, R.J., & Athanasopoulos, G. (2021). "Forecasting: principles and practice, 3rd edition: 
Chapter 11: Forecasting hierarchical and grouped series.". OTexts: Melbourne, Australia. OTexts.com/fpp3 
Accessed on July 2022.](https://otexts.com/fpp3/hierarchical.html)<br>
- [Orcutt, G.H., Watts, H.W., & Edwards, J.B.(1968). Data aggregation and information loss. The American 
Economic Review, 58 , 773{787).](http://www.jstor.org/stable/1815532)<br>
- [Disaggregation methods to expedite product line forecasting. Journal of Forecasting, 9 , 233–254. 
doi:10.1002/for.3980090304.](https://onlinelibrary.wiley.com/doi/abs/10.1002/for.3980090304)<br>
- [Wickramasuriya, S. L., Athanasopoulos, G., & Hyndman, R. J. (2019). \"Optimal forecast reconciliation for
hierarchical and grouped time series through trace minimization\". Journal of the American Statistical Association, 
114 , 804–819. doi:10.1080/01621459.2018.1448825.](https://robjhyndman.com/publications/mint/)<br>
- [Ben Taieb, S., & Koo, B. (2019). Regularized regression for hierarchical forecasting without 
unbiasedness conditions. In Proceedings of the 25th ACM SIGKDD International Conference on Knowledge 
Discovery & Data Mining KDD '19 (p. 1337{1347). New York, NY, USA: Association for Computing Machinery.](https://doi.org/10.1145/3292500.3330976)<br>